In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv('D:/my_ML/diploma_polytech/data/raw/vehicle_ins_data_1.csv', sep = ";",index_col= False)

# Создание целевой переменной: 1 если N_claims_year > 1, иначе 0
data['claim_prob'] = (data['N_claims_year'] > 1).astype(int)

# Предобработка данных
# Удаление ненужных столбцов, но НЕ УДАЛЯЕМ Date_start_contract
cols_to_drop = ['ID', 'Date_last_renewal', 'Date_next_renewal', 
                'Date_lapse', 'N_claims_year', 'Cost_claims_year', 'N_claims_history', 
                'R_Claims_history']
data = data.drop(columns=cols_to_drop)

# Преобразуем Date_start_contract в datetime ДО любой обработки
data['Date_start_contract'] = pd.to_datetime(data['Date_start_contract'], errors='coerce')

C:\Users\andre\AppData\Local\Temp\ipykernel_1724\3855888060.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('D:/my_ML/diploma_polytech/data/raw/vehicle_ins_data_1.csv', sep = ";",index_col= False)


In [4]:
# 1. Сначала обработать ВСЕ датовые колонки, если они существуют
date_columns = ['Date_birth', 'Date_driving_licence']

# Создаем список существующих датовых колонок
existing_date_columns = [col for col in date_columns if col in data.columns]

# Проверим, какие датовые колонки нашлись
print("Существующие датовые колонки для обработки:", existing_date_columns)

if not existing_date_columns:
    print("Предупреждение: Датовые колонки не найдены в DataFrame.")
else:
    for col in existing_date_columns:
        data[col] = pd.to_datetime(data[col], format='%d/%m/%Y', errors='coerce')

    # 2. Извлечь числовые признаки из существующих дат
    reference_date = pd.to_datetime('2019-12-31')
    
    if 'Date_birth' in data.columns:
        data['Age'] = (reference_date - data['Date_birth']).dt.days // 365
    
    if 'Date_driving_licence' in data.columns:
        data['Driving_experience'] = (reference_date - data['Date_driving_licence']).dt.days // 365

    

# 2. Извлечь числовые признаки из дат
reference_date = pd.to_datetime('2019-12-31')
data['Age'] = (reference_date - data['Date_birth']).dt.days // 365
data['Driving_experience'] = (reference_date - data['Date_driving_licence']).dt.days // 365

# 3. Удалить исходные датовые колонки
data = data.drop(columns=date_columns)
data = data.drop(columns=existing_date_columns, errors='ignore')
# 4. Обработать категориальные переменные
categorical_cols = ['Type_fuel']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

# 5. Убедиться, что все данные числовые
data = data.apply(pd.to_numeric, errors='coerce')

# 6. Обработка пропущенных значений
imputer = SimpleImputer(strategy='median')
data_imputed = imputer.fit_transform(data)
data = pd.DataFrame(data_imputed, columns=data.columns)

Существующие датовые колонки для обработки: ['Date_birth', 'Date_driving_licence']


In [5]:
# Разделение на признаки и целевую переменную
X = data.drop(columns=['claim_prob'])
y = data['claim_prob']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Преобразование в тензоры PyTorch
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [6]:
class TorchKGPolyModel(nn.Module):
    """
    Реализация элементарного образа полинома Колмогорова-Габора на PyTorch
    Соответствует формулам (30)-(31) из статьи
    """
    
    def __init__(self, input_size, degree=3):
        super().__init__()
        self.degree = degree
        self.linear_layer = nn.Linear(input_size, 1)  # Линейная свертка
        self.poly_coefs = nn.Parameter(torch.ones(degree + 1))  # Коэффициенты полинома
        
    def forward(self, x):
        # Этап 1: Линейная свертка (y')
        y_prime = self.linear_layer(x)
        
        # Этап 2: Полиномиальная настройка
        poly_features = torch.cat([y_prime**i for i in range(self.degree + 1)], dim=1)
        output = torch.matmul(poly_features, self.poly_coefs.unsqueeze(1))
        
        return torch.sigmoid(output)



In [11]:
# Обучение PyTorch версии
torch_kg_model = TorchKGPolyModel(input_size=X_train_tensor.shape[1], degree=3)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(torch_kg_model.parameters(), lr=0.01)

# Обучение
epochs = 300
for epoch in range(epochs):
    torch_kg_model.train()
    optimizer.zero_grad()
    
    outputs = torch_kg_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')



Epoch [20/300], Loss: 1.0770
Epoch [40/300], Loss: 0.8869
Epoch [60/300], Loss: 0.6788
Epoch [80/300], Loss: 0.4609
Epoch [100/300], Loss: 0.3187
Epoch [120/300], Loss: 0.2941
Epoch [140/300], Loss: 0.2944
Epoch [160/300], Loss: 0.2939
Epoch [180/300], Loss: 0.2939
Epoch [200/300], Loss: 0.2939
Epoch [220/300], Loss: 0.2939
Epoch [240/300], Loss: 0.2939
Epoch [260/300], Loss: 0.2939
Epoch [280/300], Loss: 0.2939
Epoch [300/300], Loss: 0.2939


In [12]:
# Оценка PyTorch модели
torch_kg_model.eval()
with torch.no_grad():
    y_pred_torch_proba = torch_kg_model(X_test_tensor).numpy()

roc_auc_torch = roc_auc_score(y_test, y_pred_torch_proba)
print(f"ROC-AUC (PyTorch K-G): {roc_auc_torch:.4f}")

ROC-AUC (PyTorch K-G): 0.7047
